<a href="https://colab.research.google.com/github/krakowiakpawel9/neural-network-course/blob/master/07_rnn/02_text_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* @author: krakowiakpawel9@gmail.com  
* @site: e-smartdata.org

In [0]:
# Update: Należy użyć biblioteki tensorflow w wersji 2.0
!pip install -q tensorflow==2.0

In [0]:
# %tensorflow_version 2.x
import numpy as np
import os

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

TensorFlow 2.x selected.


In [0]:
!wget https://storage.googleapis.com/esmartdata-courses-files/ann-course/reviews.zip
!unzip -q reviews.zip

--2019-11-18 14:20:19--  https://storage.googleapis.com/esmartdata-courses-files/ann-course/reviews.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.195.128, 2607:f8b0:400e:c01::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.195.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42878657 (41M) [application/x-zip-compressed]
Saving to: ‘reviews.zip’

reviews.zip         100%[===================>]  40.89M   159MB/s    in 0.3s    

2019-11-18 14:20:19 (159 MB/s) - ‘reviews.zip’ saved [42878657/42878657]



In [0]:
data_dir = './reviews'
train_dir = os.path.join(data_dir, 'train')

train_texts = []
train_labels = []

for label_type in ['neg', 'pos']:
    dir_name = os.path.join(train_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname))
            train_texts.append(f.read())
            f.close()
            if label_type == 'neg':
                train_labels.append(0)
            else:
                train_labels.append(1)

In [0]:
test_dir = os.path.join(data_dir, 'test')

test_texts = []
test_labels = []

for label_type in ['neg', 'pos']:
    dir_name = os.path.join(test_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname))
            test_texts.append(f.read())
            f.close()
            if label_type == 'neg':
                test_labels.append(0)
            else:
                test_labels.append(1)

In [0]:
train_texts[:10]

['You might be a bit confused if you watch this silly made-for from the beginning, since the credits list Susan Dey as "Special Guest Star." Um, why would a one-off MOW like this have a guest star? Well, if you stick with it, you\'ll find yourself paying attention to little else but Ms. Dey\'s butt, wiggling in a flowered bikini as the "Partridge Family" house babe frolics on the beach to which that imaginative title alludes. Susan\'s derrière is especially compelling when she shakes it at the camera while teasing and tickling her pseudo-disaffected brother in one mildly incestuous scene. Sadly, Susie and her tush fight a losing battle: the jiggle-TV craze that might have put that bottom over the top was three years off, so that sweet booty just gets a supporting role. In 1976 Fat Freddy Silverman would have put that behind right out front and used this flick as Susan\'s audition tape for "Charlie\'s Angels." As is, our Susan was denied cheesecake immortality and had to settle for a ve

In [0]:
train_labels[:10]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [0]:
train_labels[-10:]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [0]:
maxlen = 100   # skracamy recenzje do 100 słów
num_words = 10000    # 10000 najczęściej pojawiających się słów
embedding_dim = 100

tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(train_texts)

In [0]:
list(tokenizer.index_word.items())[:20]

[(1, 'the'),
 (2, 'and'),
 (3, 'a'),
 (4, 'of'),
 (5, 'to'),
 (6, 'is'),
 (7, 'br'),
 (8, 'in'),
 (9, 'it'),
 (10, 'i'),
 (11, 'this'),
 (12, 'that'),
 (13, 'was'),
 (14, 'as'),
 (15, 'for'),
 (16, 'with'),
 (17, 'movie'),
 (18, 'but'),
 (19, 'film'),
 (20, 'on')]

In [0]:
sequences = tokenizer.texts_to_sequences(train_texts)
print(sequences[:3])

[[22, 235, 27, 3, 224, 1467, 44, 22, 103, 11, 705, 90, 15, 36, 1, 451, 234, 1, 894, 1025, 2781, 14, 315, 3462, 320, 6985, 135, 59, 3, 28, 122, 37, 11, 25, 3, 3462, 320, 70, 44, 22, 1226, 16, 9, 487, 166, 620, 2640, 688, 5, 114, 331, 18, 1559, 3606, 8, 3, 6702, 14, 1, 220, 310, 4517, 20, 1, 2576, 5, 60, 12, 3264, 422, 6, 258, 1441, 51, 56, 9654, 9, 30, 1, 367, 134, 2, 38, 3899, 593, 8, 28, 2782, 7693, 133, 1035, 2, 38, 545, 3, 2321, 982, 1, 245, 12, 235, 25, 273, 12, 1321, 117, 1, 347, 13, 286, 150, 122, 35, 12, 1043, 40, 211, 3, 693, 214, 8, 5674, 1918, 2278, 7476, 59, 25, 273, 12, 493, 205, 43, 1005, 2, 340, 11, 506, 14, 6192, 2211, 15, 7262, 3036, 14, 6, 260, 2781, 13, 9259, 2, 66, 5, 4174, 15, 3, 52, 7576, 608, 392, 9878, 5924, 369, 7, 7, 1, 646, 721, 9, 212, 27, 298, 6, 21, 52, 1441, 1, 1381, 6, 324, 2, 3537, 8, 3, 1158, 93, 12, 3348, 6792, 22, 80, 533, 47, 235, 27, 46, 2798, 1515, 1656, 30, 154, 493, 1, 367, 5844, 42, 40, 3, 3463, 833, 5229, 1693, 90, 15, 245, 706, 12, 1664, 7577,

In [0]:
word_index = tokenizer.word_index
print(f'{len(word_index)} unikatowych słów.')

88582 unikatowych słów.


In [0]:
# skracamy recenzje do pierwszych 100 słów
train_data = pad_sequences(sequences, maxlen=maxlen)
train_data.shape

(25000, 100)

In [0]:
train_data[:3]

array([[  15, 1134,    8,   11,   17,    6,    8,    1, 3538, 1665,  489,
         106,    4, 1227,  253,   31,    1,  193, 1547,   44,  123, 2024,
          34,    6,  420,    5,   27, 6793,    2, 3196,    2,  235,  231,
          46, 2502,   15,    2, 3210,    5,    1, 3008,   12,    6, 1500,
        3539,   24,  220,   18, 1227,   14,  253,   31, 7477,  135,  158,
          72,   64,   50,   36,   11,    6,  162,   40, 9655,    2, 4348,
           2,  215,   28,  224, 8595,   51,  191, 1092,  211,  618,    2,
        3765,    3,  344,    8,    1, 9457,    2, 6354, 6436,   15,    1,
        1657,    2, 2781,   47,  212,   27,   46,    4,   22,   43,   47,
          61],
       [ 676,    3, 1560, 7364,  964,   67, 1132,  177,    1, 1314,  206,
         256,   18,   11,    6,  797,    4,    3,   19,  118, 6703, 4588,
         288, 6794,  567,   65,  516,  177,  462, 9093,   15,    3,    2,
          81,  757,   65,  319,    6, 2736, 1861, 6987,    5,  373,    1,
         565,   12,  22

In [0]:
train_labels = np.asarray(train_labels)
train_labels

array([0, 0, 0, ..., 1, 1, 1])

In [0]:
# przemieszanie próbek
indices = np.arange(train_data.shape[0])
np.random.shuffle(indices)
train_data = train_data[indices]
train_labels = train_labels[indices]

train_data.shape

(25000, 100)

In [0]:
# podział na zbiór treningowy i walidacyjny
training_samples = 15000
validation_samples = 10000

X_train = train_data[:training_samples]
y_train = train_labels[:training_samples]
X_val = train_data[training_samples: training_samples + validation_samples]
y_val = train_labels[training_samples: training_samples + validation_samples]

In [0]:
# budowa modelu
# Embedding(input_dim, output_dim)

model = Sequential()
model.add(Embedding(num_words, embedding_dim, input_length=maxlen))
model.add(Flatten())
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 100)          1000000   
_________________________________________________________________
flatten (Flatten)            (None, 10000)             0         
_________________________________________________________________
dense (Dense)                (None, 16)                160016    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 1,160,033
Trainable params: 1,160,033
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [0]:
history = model.fit(X_train, y_train, batch_size=32, epochs=5, validation_data=(X_val, y_val))

Train on 15000 samples, validate on 10000 samples
Epoch 1/5
15000/15000 [==============================] - 9s 625us/sample - loss: 0.4631 - accuracy: 0.7719 - val_loss: 0.3457 - val_accuracy: 0.8472
Epoch 2/5
15000/15000 [==============================] - 8s 533us/sample - loss: 0.1572 - accuracy: 0.9419 - val_loss: 0.3943 - val_accuracy: 0.8361
Epoch 3/5
15000/15000 [==============================] - 8s 532us/sample - loss: 0.0190 - accuracy: 0.9963 - val_loss: 0.5358 - val_accuracy: 0.8301
Epoch 4/5
15000/15000 [==============================] - 8s 535us/sample - loss: 0.0016 - accuracy: 0.9995 - val_loss: 0.7097 - val_accuracy: 0.8282
Epoch 5/5
15000/15000 [==============================] - 8s 519us/sample - loss: 2.8003e-05 - accuracy: 1.0000 - val_loss: 0.8854 - val_accuracy: 0.8292


In [0]:
def plot_hist(history):
    import pandas as pd
    import plotly.graph_objects as go
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['accuracy'], name='accuracy', mode='markers+lines'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_accuracy'], name='val_accuracy', mode='markers+lines'))
    fig.update_layout(width=1000, height=500, title='accuracy vs. val accuracy', xaxis_title='Epoki', yaxis_title='accuracy', yaxis_type='log')
    fig.show()

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['loss'], name='loss', mode='markers+lines'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_loss'], name='val_loss', mode='markers+lines'))
    fig.update_layout(width=1000, height=500, title='loss vs. val loss', xaxis_title='Epoki', yaxis_title='loss', yaxis_type='log')
    fig.show()

plot_hist(history)

In [0]:
sequences = tokenizer.texts_to_sequences(test_texts)
X_test = pad_sequences(sequences, maxlen=maxlen)
y_test = np.asarray(test_labels)

model.evaluate(X_test, y_test, verbose=0)

### Simple RNN

In [0]:
from tensorflow.keras.layers import SimpleRNN, LSTM 

In [0]:
model = Sequential()
model.add(Embedding(10000, 32))
model.add(SimpleRNN(16))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 32)          320000    
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 16)                784       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total params: 320,801
Trainable params: 320,801
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [0]:
history = model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_val, y_val))

Train on 15000 samples, validate on 10000 samples
Epoch 1/10
15000/15000 [==============================] - 17s 1ms/sample - loss: 0.5522 - accuracy: 0.7119 - val_loss: 0.4367 - val_accuracy: 0.8050
Epoch 2/10
15000/15000 [==============================] - 15s 1ms/sample - loss: 0.3389 - accuracy: 0.8607 - val_loss: 0.3663 - val_accuracy: 0.8396
Epoch 3/10
15000/15000 [==============================] - 15s 977us/sample - loss: 0.2612 - accuracy: 0.8969 - val_loss: 0.3789 - val_accuracy: 0.8440
Epoch 4/10
15000/15000 [==============================] - 15s 981us/sample - loss: 0.1926 - accuracy: 0.9290 - val_loss: 0.3829 - val_accuracy: 0.8411
Epoch 5/10
15000/15000 [==============================] - 15s 993us/sample - loss: 0.1367 - accuracy: 0.9512 - val_loss: 0.5065 - val_accuracy: 0.8168
Epoch 6/10
15000/15000 [==============================] - 15s 994us/sample - loss: 0.0932 - accuracy: 0.9683 - val_loss: 0.4786 - val_accuracy: 0.8258
Epoch 7/10
15000/15000 [========================

In [0]:
plot_hist(history)

In [0]:
model = Sequential()
model.add(Embedding(10000, 32))
model.add(LSTM(16))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 32)          320000    
_________________________________________________________________
lstm (LSTM)                  (None, 16)                3136      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 323,153
Trainable params: 323,153
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [0]:
history = model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_val, y_val))

Train on 15000 samples, validate on 10000 samples
Epoch 1/10
15000/15000 [==============================] - 25s 2ms/sample - loss: 0.4554 - accuracy: 0.7850 - val_loss: 0.3474 - val_accuracy: 0.8510
Epoch 2/10
15000/15000 [==============================] - 22s 1ms/sample - loss: 0.2863 - accuracy: 0.8827 - val_loss: 0.3447 - val_accuracy: 0.8497
Epoch 3/10
15000/15000 [==============================] - 22s 1ms/sample - loss: 0.2396 - accuracy: 0.9056 - val_loss: 0.3445 - val_accuracy: 0.8466
Epoch 4/10
15000/15000 [==============================] - 22s 1ms/sample - loss: 0.2086 - accuracy: 0.9189 - val_loss: 0.3693 - val_accuracy: 0.8504
Epoch 5/10
15000/15000 [==============================] - 23s 2ms/sample - loss: 0.1877 - accuracy: 0.9301 - val_loss: 0.3724 - val_accuracy: 0.8503
Epoch 6/10
15000/15000 [==============================] - 22s 1ms/sample - loss: 0.1719 - accuracy: 0.9386 - val_loss: 0.4050 - val_accuracy: 0.8487
Epoch 7/10
15000/15000 [==============================] 

In [0]:
plot_hist(history)

In [0]:
model = Sequential()
model.add(Embedding(10000, 32))
model.add(LSTM(16))
model.add(Dense(1, activation='sigmoid'))
model.summary()

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, y_train, batch_size=32, epochs=3, validation_data=(X_val, y_val))

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 32)          320000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 16)                3136      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 17        
Total params: 323,153
Trainable params: 323,153
Non-trainable params: 0
_________________________________________________________________
Train on 15000 samples, validate on 10000 samples
Epoch 1/3
15000/15000 [==============================] - 25s 2ms/sample - loss: 0.4639 - accuracy: 0.7807 - val_loss: 0.4029 - val_accuracy: 0.8248
Epoch 2/3
15000/15000 [==============================] - 23s 2ms/sample - loss: 0.2900 - accuracy: 0.8810 - val_loss: 0.3347 - val_accuracy: 0.8563
Epoch 3/3
15000/15000 [=========

In [0]:
sequences = tokenizer.texts_to_sequences(test_texts)
X_test = pad_sequences(sequences, maxlen=maxlen)
y_test = np.asarray(test_labels)

model.evaluate(X_test, y_test, verbose=0)

[0.3625735130405426, 0.8492]